# Preprocess the dropseq and smFISH data

Data: https://drive.google.com/drive/folders/1uos3DDymyrh3ZyxveDhqQIcOyD1brxeL

In [2]:
import pandas as pd
import seaborn as sns
import scanpy as sc
import scipy.sparse as sparse
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as opt
import itertools
import scanpy as sc

### Process DropSeq data

In [3]:
data_path = '/data_volume/memento/saver/'

partial = pd.read_csv('/data_volume/memento/saver/melanoma_dropseq.csv', index_col=0, sep=',').T
dropseq = pd.read_csv('GSE99330_dropseqRPM.txt', sep=' ').T.loc[partial.index]/1e6

# Reverse calculate cell sizes
high_expr = partial.mean(axis=0).sort_values().tail(20).index.tolist()
dropseq_sf = (partial[high_expr]/dropseq[high_expr]).mean(axis=1)
dropseq = dropseq*dropseq_sf.values.reshape(-1,1)

# Create sparse matrix and save
X = sparse.csr_matrix(dropseq.values)
dropseq_adata = sc.AnnData(X=X, obs=pd.DataFrame(index=dropseq.index), var=pd.DataFrame(index=dropseq.columns))

dropseq_adata.write(data_path + 'full_dropseq.h5ad')

### Compute number of overlap genes

In [4]:
smfish = pd.read_csv('/data_volume/memento/saver/fishSubset.txt', index_col=0, sep=' ')
# smfish = smfish[smfish['GAPDH'] > 0]
fish_gapdh_sf = (smfish['GAPDH']+1).values
# smfish_normalized = smfish#/fish_gapdh_sf.reshape(-1, 1)
# smfish = pd.read_csv('fishSubset (1).txt', sep=' ', index_col=0)

overlap_genes = list(set(dropseq.columns) & set(smfish.columns))
overlap_genes = dropseq.mean(axis=0)[overlap_genes][dropseq.mean(axis=0)[overlap_genes] > 0].index.tolist()

print(len(overlap_genes))

# smfish = smfish[overlap_genes].fillna(0.0)

26


In [34]:
import pandas as pd
import scanpy as sc
import seaborn as sns
import scanpy as sc
import scipy.sparse as sparse
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as opt
import itertools

DATA_PATH = '/home/ubuntu/Data/'
MIN_MEAN_THRESH = 0.001

if __name__ == '__main__':
    
#     LEGACY CODE, ONLY FOR REVERSE ENGINEERING N_COUNTS
#     data_path = '/data_volume/memento/saver/'

#     partial = pd.read_csv('/data_volume/memento/saver/melanoma_dropseq.csv', index_col=0, sep=',').T
#     dropseq = pd.read_csv('GSE99330_dropseqRPM.txt', sep=' ').T.loc[partial.index]/1e6

#     # Reverse calculate cell sizes
#     high_expr = partial.mean(axis=0).sort_values().tail(20).index.tolist()
#     dropseq_sf = (partial[high_expr]/dropseq[high_expr]).mean(axis=1)
#     dropseq = dropseq*dropseq_sf.values.reshape(-1,1)

#     # Create sparse matrix and save
#     X = sparse.csr_matrix(dropseq.values)
#     dropseq_adata = sc.AnnData(X=X, obs=pd.DataFrame(index=dropseq.index), var=pd.DataFrame(index=dropseq.columns))

#     dropseq_adata.write(data_path + 'full_dropseq.h5ad')

    # Calculate stuff for Dropseq data
    dropseq_adata = sc.read_h5ad(DATA_PATH + 'smfish/full_dropseq.h5ad')
    dropseq_adata = dropseq_adata[dropseq_adata.X.sum(axis=1).A1 > 0].copy()

    dropseq_adata.obs['n_counts'] = dropseq_adata.X.sum(axis=1).A1
    dropseq_adata.obs['n_genes'] = (dropseq_adata.X > 0).sum(axis=1).A1

    z_means = dropseq_adata.X.mean(axis=0).A1    
    dropseq_genes = dropseq_adata.var.index[z_means > MIN_MEAN_THRESH].tolist()
    
    dropseq_adata.obs.index = [f'cell{i}' for i in range(dropseq_adata.shape[0])]
    dropseq_adata.var.index = [f'gene{i}' for i in range(dropseq_adata.shape[1])]
    dropseq_adata.write(DATA_PATH + 'smfish/filtered_dropseq.h5ad')

    # Calculate stuff for smFISH
    smfish = pd.read_csv(DATA_PATH + 'smfish/fishSubset.txt', index_col=0, sep=' ')
    filtered_fish = smfish.query('GAPDH > 0')
    overlap_genes = list(set(dropseq_genes) & set(smfish.columns))

    mean_genes = overlap_genes
    var_genes = [i for i in overlap_genes if i != 'GAPDH']
    corr_genes = [(a,b) for a,b in itertools.combinations(overlap_genes, 2) if 'GAPDH' not in [a,b]]

    smfish_means = np.zeros(len(mean_genes))
    smfish_variances = np.zeros(len(var_genes))
    smfish_correlations = np.zeros(len(corr_genes))

    for idx, gene in enumerate(mean_genes):
        if gene == 'GAPDH':
            smfish_means[idx] = 1.0
        df = filtered_fish[['GAPDH', gene]].dropna()
        norm = df[gene].values/df['GAPDH'].values
        smfish_means[idx] = norm.mean()

    for idx, gene in enumerate(var_genes):

        df = filtered_fish[['GAPDH', gene]].dropna()
        norm = df[gene].values/df['GAPDH'].values
        smfish_variances[idx] = norm.var()

    for idx, pair in enumerate(corr_genes):

        gene1, gene2 = pair        
        df = filtered_fish[[gene1, gene2, 'GAPDH']].dropna()

        if df.shape[0] < 2:
            smfish_correlations[idx] = np.nan
            continue
        norm1 = (df[gene1]/df['GAPDH']).values
        norm2 = (df[gene2]/df['GAPDH']).values
        smfish_correlations[idx] = stats.pearsonr(norm1, norm2)[0]


    np.savez_compressed(
        DATA_PATH + 'smfish/smfish_estimates',
        mean_genes=np.array(mean_genes),
        var_genes=np.array(var_genes),
        corr_genes = np.array(corr_genes),
        mean=smfish_means,
        variance=smfish_variances,
        correlation=smfish_correlations)

/home/ubuntu/anaconda3/envs/mementocxg/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


In [35]:
len(overlap_genes)

21

In [53]:
pd.DataFrame(zip(corr_genes, smfish_correlations)).dropna()

,0,1
0,"(EGFR, JUN)",0.130705
1,"(EGFR, TXNRD1)",0.202708
2,"(EGFR, FOSL1)",0.680911
4,"(EGFR, NRG1)",0.124664
5,"(EGFR, FGFR1)",0.238469
...,...,...
178,"(MITF, PDGFC)",0.014529
183,"(LMNA, BABAM1)",0.483998
184,"(CCNA2, WNT5A)",0.228920
185,"(CCNA2, PDGFC)",0.400606


In [45]:
df

,EGFR,KDM5A,GAPDH


In [43]:
stats.pearsonr(norm1, norm2)

PearsonRResult(statistic=0.1307045161125206, pvalue=1.2813205922984012e-52)

In [31]:
pair

('GAPDH', 'EGFR')

In [29]:
smfish_variances

array([0.00000000e+00, 7.31290172e-03, 4.24617774e+00, 1.78020778e-02,
       1.20172165e+00, 2.73404700e-03, 2.70068593e-02, 2.17340827e+00,
       4.78917560e-02, 4.25429698e+00, 3.93790448e-01, 9.85493079e-05,
       1.36325437e-04, 1.55479601e+00, 1.05530039e-01, 1.86032968e-04,
       7.13128518e-04])

In [28]:
smfish_means

array([1.00000000e+00, 8.34838178e-03, 3.59547728e-01, 1.92663877e-02,
       1.38625197e-01, 3.60623468e-03, 1.89297276e-02, 3.67053443e-01,
       4.08173383e-02, 4.91185397e-01, 1.38445272e-01, 1.44681763e-02,
       7.03048087e-04, 1.91472430e-01, 4.37495963e-02, 1.32369135e-02,
       9.62544798e-03])

In [22]:
m

0.008348381779742642

In [23]:
v

0.007312901716916828

In [19]:
norm

array([0.        , 0.00414938, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [17]:
df[gene].values/df['GAPDH'].values

array([0.        , 0.00414938, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [7]:
filtered_fish['GAPDH'].shape

(87885,)

In [8]:
filtered_fish['GAPDH'].dropna().shape

(87885,)

In [9]:
smfish['GAPDH'].shape

(88040,)

In [10]:
smfish['GAPDH'].dropna().shape

(88040,)